In [1]:
!pip install openai

In [2]:
import json
import openai

In [3]:
with open('GPT_SECRET_KEY.json') as f:
    data = json.load(f)
openai.api_key = data["API_KEY"]

In [4]:
from gpt import GPT
from gpt import Example
import numpy as np

In [5]:
gpt = GPT(engine = "davinci", temperature = 0.5, max_tokens = 100, output_prefix = "output (positive/neutral/negative):")

In [6]:
def change_labels(num):
    if num == 1:
        return "positive"
    elif num == 0:
        return "neutral"
    elif num == -1:
        return "negative"

In [7]:
import pandas as pd
def transform_txt(filename, n): # n is the number of input instances wanted for each sentiment category
    """
    This function reads input sentences and associated sentiments 
    """
    # reading text filename into a Dataframe
    data = pd.read_csv(filename, header = 0, encoding = "utf8", sep = ":->")
    df_used = data.groupby('Sentiment_class_label').head(n).reset_index(drop = True)
    df_used['Sentiment_class_label'] = df_used['Sentiment_class_label'].apply(lambda x: change_labels(x))
    return df_used
    

In [8]:
df_used = transform_txt("sm_text_sentiment_training.txt", 20)
print(df_used)

   Sentiment_class_label                                        Phrase_text
0               positive                   “kindness and compassion” [NAME]
1               positive  “That’s also cute” I say as I look at the 5k i...
2               positive  “I have no experience being mean” is probably ...
3               positive  “I am on the verge of exploding in my pants” “...
4               positive                                “Enjoy your dinner”
5               positive  “But [NAME] helped the Browns win more games t...
6               positive                                               zeal
7               positive                                           youthful
8               positive                               youre awesome buddy!
9               positive                            your voice is beautiful
10              positive                            your vehicle is pretty!
11              positive                                     your beautiful
12          

In [9]:
def add_examples(gpt_instance, df_used, n): # n is the number of Example instances to "train" GPT-3 on
    df_subset = df_used.groupby('Sentiment_class_label').head(n).reset_index(drop = True)
    for row in range(df_subset.shape[0]):
        gpt_instance.add_example(Example(df_subset['Phrase_text'][row], df_subset['Sentiment_class_label'][row]))
    return gpt_instance

In [10]:
gpt = add_examples(gpt, df_used, 3)

In [11]:
# A function to write prompt into GPT-3 API:
def write_prompts(df_used, gpt_instance):
    df_used['gpt_output'] = df_used["Phrase_text"].apply(lambda x: gpt_instance.submit_request(x).choices[0].text.replace("output (positive/neutral/negative):", "").strip("\n"))
    df_used['matched'] = np.where(df_used['Sentiment_class_label'] == df_used['gpt_output'], 1, 0)
    # dropping Sentiment_class_label column
    df_used.drop(['Sentiment_class_label'], axis = 1)
    return df_used
    
    


In [12]:
out_df = write_prompts(df_used, gpt)

In [13]:
print(out_df)

   Sentiment_class_label                                        Phrase_text  \
0               positive                   “kindness and compassion” [NAME]   
1               positive  “That’s also cute” I say as I look at the 5k i...   
2               positive  “I have no experience being mean” is probably ...   
3               positive  “I am on the verge of exploding in my pants” “...   
4               positive                                “Enjoy your dinner”   
5               positive  “But [NAME] helped the Browns win more games t...   
6               positive                                               zeal   
7               positive                                           youthful   
8               positive                               youre awesome buddy!   
9               positive                            your voice is beautiful   
10              positive                            your vehicle is pretty!   
11              positive                            

In [14]:
out_df.drop("Sentiment_class_label", axis = 1, inplace = True)

In [15]:
accuracy = ((np.sum(out_df['matched'])) / (out_df.shape[0])) * 100
print(accuracy)

68.33333333333333
